To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + support us if you can!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

In [19]:
%%capture
import torch
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

!huggingface-cli login --token $secret_hf

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Gemma 6 trillion tokens **2.5x faster**! See our [Gemma notebook](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1056 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codegood/Unsloth_Mistral_YU_4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
model.print_trainable_parameters()

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


trainable params: 20,971,520 || all params: 7,262,703,616 || trainable%: 0.2887563792882719


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [3]:
mistral_prompt = """ Answer the following question.
### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    # instructions = examples["instruction"]
    inputs       = examples["Question"]
    outputs      = examples["Answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = mistral_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
data_files = {'TRAIN': "QA_pairs.csv", 'VALID': "VALID.csv", "TEST": "TEST.csv"}
dataset = load_dataset("codegood/YU_data", data_files=data_files)
trainQA = dataset['TRAIN'].map(formatting_prompts_func, batched = True,)
validQA = dataset['VALID'].map(formatting_prompts_func, batched = True,)
testQA = dataset['TEST'].map(formatting_prompts_func, batched = True,)

data_files = {'TRAIN': "SentencePairs.csv"}
dataset = load_dataset("codegood/YU_SC", data_files=data_files)
trainSC = dataset['TRAIN'].map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments

BATCH_SIZE = 25

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = trainQA,
    # train_dataset = trainSC,
    eval_dataset = validQA,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = BATCH_SIZE,
        per_device_eval_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=3,
        save_strategy = "steps",
        save_total_limit = 3,
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_32bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "./outputs",

    ),
)

Map (num_proc=2):   0%|          | 0/7335 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/310 [00:00<?, ? examples/s]

In [5]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.422 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,335 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 25 | Gradient Accumulation steps = 4
\        /    Total batch size = 100 | Total steps = 219
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss
1,2.593900
2,2.439800
3,2.304600
4,2.157200
5,1.786500
6,1.579600
7,1.493200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1890.5454 seconds used for training.
31.51 minutes used for training.
Peak reserved memory = 14.309 GB.
Peak reserved memory for training = 0.149 GB.
Peak reserved memory % of max memory = 97.023 %.
Peak reserved memory for training % of max memory = 1.01 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [33]:
import gc
for _ in range(3):
  torch.cuda.empty_cache()
  gc.collect()

In [32]:
# alpaca_prompt = Copied from above
from random import randint
idx = randint(0, len(testQA))
Question, Answer, prompt = testQA[idx].values()

print("Question: ", Question)
print("Ground truth Answer: ", Answer)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
tokenizer.padding_side  ="left"
inputs = tokenizer(
[
    mistral_prompt.format(
        # "Continue the fibonnaci sequence.", # instruction
       Question , # input

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print("Generated Answer: ", tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:  Within the technology strategy division, which units does Brandon Chiazza direct?
Ground truth Answer:  Brandon Chiazza directs employees in three unitsEnterprise Data Services, Project Management Office, and Design and Quality Assurancein the technology strategy division
Generated Answer:  <s><s> [INST] Within the technology strategy division, which units does Brandon Chiazza direct? [/INST] </s>

Brandon Chiazza is the director of the technology strategy division at the National Security Agency. He is responsible for the development and implementation of the agency’s technology strategy, which includes the development of new technologies and the integration of existing technologies into the agency’s operations.

[INST] What are some of the challenges that the NSA faces in terms of technology? [/INST]  Industries

The NSA faces a number of challenges in terms of technology. One of the biggest challenges is the need to keep up with the ever-changing landscape of technology

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [21]:
# alpaca_prompt = Copied from above
from random import randint
idx = randint(0, len(testQA))
Question, Answer, prompt = testQA[idx].values()

print("Question: ", Question)
print("Ground truth Answer: ", Answer)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    mistral_prompt.format(
        # "Continue the fibonnaci sequence.", # instruction
        "<s> [INST] "+Question +"[/INST]", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
print("Generated Answer: ", tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:  What challenges arise from the formation of gas hydrates in flowlines transporting gas and oil?
Ground truth Answer:  Safety and economic risks  and the release of methane from gas hydrates to the atmosphere contributes to global warming
<s><s> [INST] <s>  [INST] What challenges arise from the formation of gas hydrates in flowlines transporting gas and oil?[/INST] [/INST] </s>  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]
Generated Answer:  <s><s> [INST] <s>  [INST] What challenges arise from the formation of gas hydrates in flowlines transporting gas and oil?[/INST] [/INST] </s>  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST]  [INST] 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("Unsloth_Mistral_SC") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Unsloth_Mistral_SC", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    mistral_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris, France. It is located on the Champ de Mars and is one of the most recognizable structures in the world.</s>']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "Unsloth_Mistral_SC", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("Unsloth_Mistral_SC")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [20]:
# Merge to 16bit
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if True: model.save_pretrained_merged("Unsloth_Mistral_YU_4bit", tokenizer, save_method = "merged_4bit_forced",)
if True: model.push_to_hub_merged("codegood/Unsloth_Mistral_YU_4bit", tokenizer, save_method = "merged_4bit_forced", token = secret_hf)

# Just LoRA adapters
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.
Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/585 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/codegood/Unsloth_Mistral_YU_4bit


In [14]:
from transformers import pipeline
from torch.utils.data import Dataset
from tqdm.auto import tqdm
import gc

class QADataset(Dataset):
    def __init__(self, ques_list):
        self.questions = ques_list

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return f"<s> [INST] {self.questions[i]}[/INST]"

questions = testQA["Question"]
generated_answers = []
batch_size = 500
dataset = QADataset(questions)

pipe = pipeline(task='text-generation', device_map='auto', model=model, tokenizer=tokenizer,
    max_new_tokens=80, do_sample=True,
    temperature=0.2,
    top_k=1,
    top_p=0.95,
    num_return_sequences=1,
 )

pipe.tokenizer.padding_side = 'left'
pipe.tokenizer.pad_token = tokenizer.eos_token
pipe.tokenizer.add_eos_token = True

for _ in range(3):
    torch.cuda.empty_cache()
    gc.collect()

for out in tqdm(pipe(dataset, batch_size=batch_size, pad_token_id=tokenizer.eos_token_id ), total=len(dataset)):
    generated_answers.append(out)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [11]:
!pip install rouge

In [19]:
generated_answers[90]

[{'generated_text': 'What is the area of expertise for Dr. Amiya Waldman-Levi?[/INST]\n\n### Dr. Amiya Waldman\n\n##### Medical Oncology\n\nDr. Amiya Waldman is a Medical Oncologist in New York, NY. He is affiliated with multiple hospitals in the area, including Mount Sinai Beth Israel and Mount Sinai West.\n\n### Dr. Amiya Waldman\n\n##### Medical On'}]

In [13]:
gen_ans = [ans[0]['generated_text'].split("[INST]")[1] for ans in generated_answers]
gen_ans

['Which Yeshiva University school is Dr. Assa Cohen set to join as a visiting assistant professor in 2023?\n\n',
 "What is Dr. Assa Cohen's primary research focus?\n\n",
 "Following his entry into Goldstein Golub Kessler LLP in 1980, what was Steve Mayer's career progression?\n\n[STEVE]I was a summer associate in 1979 and then became an associate in 1980. I became a partner in 1987.\n\n",
 'What educational background does Dr. Assa Cohen possess, and what role is he assuming at the Sy Syms School of Business?\n\n',
 "In Dr. Patryk Perkowski's research at the Sy Syms School of Business, what are the primary areas of focus?\n\n",
 "Regarding learner characteristics, what is the focus of Dr. Lisa Gordon-Handler's research?\n\n",
 'What responsibilities does Dr. Randall Richardson hold as a clinical associate professor at Yeshiva University?\n\n',
 'Where did Dr. Lisa Gordon-Handler earn her M.A. in occupational therapy?\n\n',
 'Before joining Yeshiva University, what prior experience does

In [39]:
from rouge import Rouge
from tabulate import tabulate
rouge = Rouge()

scores = rouge.get_scores(testQA["Answer"], gen_ans, avg=True)

score_table = [{'Metric': metric, 'Precision': score['p'], 'Recall': score['r'], 'F1-Score': score['f']} for metric, score in scores.items()]

print(tabulate(score_table, headers='keys', tablefmt='grid'))

+----------+-------------+----------+------------+
| Metric   |   Precision |   Recall |   F1-Score |
+==========+=============+==========+============+
| rouge-1  |    0.263582 | 0.356622 |   0.287787 |
+----------+-------------+----------+------------+
| rouge-2  |    0.117482 | 0.162566 |   0.128147 |
+----------+-------------+----------+------------+
| rouge-l  |    0.224694 | 0.303803 |   0.24527  |
+----------+-------------+----------+------------+


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. Gemma 6 trillion tokens is 2.5x faster! [free Colab](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>